In [31]:
from helpers import load_csv_data
import os
import numpy as np
import matplotlib.pyplot as plt

In [76]:
x_train, x_test, y_train, train_ids, test_ids = load_csv_data('./data/dataset/dataset', sub_sample = False)

In [146]:
# Remove columns with more than lim_col [%] of Nans
keep_col = []
lim_col = 0.2
for column in range(x_train.shape[1]):
    percentage = np.sum(np.isnan(x_train[:, column])) / x_train.shape[0]
    keep_col.append(percentage < lim_col)

In [147]:
x_tr = x_train[:, keep_col] # Reduced number of features

In [148]:
# Remove datapoints (rows) with one or mutliple Nans
keep_row = []
for row in range(x_tr.shape[0]):
    keep_row.append(np.sum(np.isnan(x_tr[row,:])) == 0)

In [149]:
x_tr = x_tr[keep_row, :] # Reduced number of datapoints